In [ ]:
__author__ = "Jose David Marroquin Toledo"
__credits__ = ["Jose David Marroquin Toledo", ]
__email__ = "jose@marroquin.cl"
__status__ = "Development"

# Fourier Ptychographic Imaging

## 1. Forward Imaging Model

In this process, the sample is illuminated with one LED of the grid at time and a camera captures Lo-Res images under different incident angles. To simulate it, we have to:

1. Create a Hi-Res complex image.

2. Generate the incident wave vectors.

3. Produce the ouput Lo-Res images.

#####  [2. The Recovery Process](phaseretrieval.ipynb)

In [ ]:
from PIL import Image
import numpy as np
import math
import matplotlib.pyplot as plt
import decimal
import scipy.misc
import os

#### 1.1. Create the Hi-Res complex image.

In [ ]:
def generate_obj(amplitude, phase, **kwargs):
    """Return the Hi-Res complex object ('numpy.ndarray'), the sample.
    """
    # Show the output image in the notebook.
    show = kwargs.pop('show', False)
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(generate_obj.__name__, list(kwargs.keys())[-1]))
    amplitude = Image.open(amplitude)
    phase = Image.open(phase)
    w, h = amplitude.size  # It is not the same that
                           # np.ndarray.shape()
    # In MATLAB, imresize uses bicubic interpolation by default.
    phase = phase.resize((w, h), resample=Image.BICUBIC)
    # 'd' ('str') is a character code for a double-precision
    # floating-point number.
    arr_amplitude = np.array(amplitude, dtype='d')
    arr_phase = np.array(phase, dtype='d')
    amplitude.close()
    phase.close()
    arr_phase = math.pi * arr_phase / np.amax(arr_phase)
    obj = arr_amplitude * np.exp(1j * arr_phase)
    obj = np.absolute(obj)  # The Hi-Res complex image.
    if show:
        # Using PIL, the objects is shown as an image almost
        # completely white.
        # img_obj = Image.fromarray(obj)
        # img_obj.show()
        # Using matplotlib anc 'Greys_r' ('str') as value for 'cmap'
        # key [1], the object is shown like in Matlab R2009b.
        #
        # [1] unutbu. (2010). Display image as grayscale using matplotlib [Msg 1]. Message posted to
        # http://stackoverflow.com/questions/3823752/display-image-as-grayscale-using-matplotlib?answertab=votes#tab-top
        plt_img = plt.imshow(obj, cmap='Greys_r')
    return obj

#### 1.2. Generate the incident wave vectors.

In [ ]:
def gen_xy_ring_led(d, ledsperring):
    """Return a list with the (x, y) coordinates of each LED of a LED
    ring.
    
    d ('int') is the distance in mm between LEDs and ledsperring
    ('int') is a list with the number of LEDs per ring from the
    center of this."""
    l_coord = list()
    l_row = list()
    for nleds in ledsperring:
        r = (d * nleds) / (2.0 * math.pi)
        theta_step = int(360 / nleds)
        range_theta = range(0, 360, theta_step)
        range_theta = range_theta[:nleds]
        for theta in range_theta:
            x = r * math.cos(math.radians(theta))
            y = r * math.sin(math.radians(theta))
            l_row.append((x, y))
        l_coord.append(l_row)
        l_row = list()
    return l_coord

In [ ]:
def gen_xy_led_grid(n, d):
    """Return a list with the (x, y) coordinates of each LED of a
    n-by-n LED grid.
    
    n ('int') is the number of LEDs per row or per column in the grid
    LED and d is the distance in mm between LEDs.
    """
    l_coord = list()
    x_max = math.floor((n / 2)) * d
    x_min = -x_max
    y_max = x_max
    y_min = x_min
    l_range = list(range(x_min, x_max + 1, d))
    l_row = list()
    for i in l_range:
        for j in l_range:
            l_row.append((j, -i))  # (x, y, z)
        l_coord.append(l_row)
        l_row = list()
    return l_coord

In [ ]:
def generate_wave_vectors(l_coord, height):
    """Return an array ('numpy.ndarray') with the magnitude of
    the x-component and y-component of the incident wave vectors that
    emerge from a LED illuminator. Each LED is represented by a 
    (x, y) tuple in l_coord ('list').
    
    height ('int') is the distance between the LED illuminator and
    the sample.
    """
    arr = np.array(l_coord)  # arr ('numpy.ndarray') contains (x, y)
                             # coordinates.
    arr = arr / height
    arr = np.arctan(arr)
    arr = np.sin(arr)
    arr = -arr
    return arr

In [ ]:
def round_half_up(num):
    """In Python 3, the round() function had changed. For example,
    round(2.5) returns 2 ('int') like round(1.5). It is possible to
    obtain 3 ('int') for 2.5 rounded using the decimal module [2].
    
    [2] Barthelemy. (2014). Python 3.x rounding behavior. Message
    posted to
    http://stackoverflow.com/questions/10825926/python-3-x-rounding-behavior
    """
    return int(decimal.Decimal(num).quantize(decimal.Decimal(1),
                                             rounding=decimal.ROUND_HALF_UP))

In [ ]:
def get_cft(wave_vectors, wavelen, ccdpx, zoom, na, w, h):
    """Return the number of rows and columns of the Hi-Res output,
    vector waves, dkx ('float') and dky ('float'), and the coherent
    transfer function of the objective.
    
    wave_vectors is a numpy.ndarray, wavelen is a int, ccdpx ('float')
    is the sampling pixel size of the CCD, hirespx ('float') is the
    pixel size of the reconstruction, na ('float') is the numerical
    aperture of the lens, w ('int') the width in pixels of the output
    Hi-Res image and h ('int') its height. zoom ('int') is a STRANGE
    PARAMETER but it acts as it were a zoom.
    """
    k_0 = 2 * math.pi / wavelen
    hirespx = ccdpx / zoom
    dkx = 2 * math.pi / (hirespx * w)
    dky = 2 * math.pi / (hirespx * h)
    p = int(h / (ccdpx / hirespx))  # Number of rows of the output.
    q = int(w / (ccdpx / hirespx))  # Number of columns of the output.
    img_seq_lores = list()
    k = k_0 * wave_vectors;
    # From all rows, all columns, extract the first element.
    kx = k[:, :, 0]
    # Reshape xk ('numpy.ndarray') in 1-D array.
    kx = np.reshape(kx, len(kx.flat))
    ky = k[:, :, 1]
    ky = np.reshape(ky, len(ky.flat))
    cutoff_freq = na * k_0
    k_max = math.pi / ccdpx
    kxm, kym = np.meshgrid(np.arange(-k_max, k_max + 1,
                                     k_max / ((q - 1) / 2)),
                           np.arange(-k_max, k_max + 1,
                                     k_max / ((p - 1) / 2)))
    coherent_transfer_funct = ((kxm ** 2 + kym ** 2) < cutoff_freq ** 2)
    # E.g., convert from [[False, True, ..., False, False],
    #                     ...,
    #                     [True, False, ..., True, True]]
    #                 to [[0, 1, ..., 0, 0],
    #                     , ...,
    #                     [1, 0, ..., 1, 1]] ('numpy.ndarray').
    coherent_transfer_funct = coherent_transfer_funct.astype(float)
    return p, q, dkx, dky, kx, ky, coherent_transfer_funct

#### 3. Produce the ouput Lo-Res images.

In [ ]:
def num_str_zeros(num, n_digs, matlab=False):
    """Return a string that contains a sequence n-zeros followed by
    num ('int') as 'str', for example, num_str_zeros(89, 4) returns
    '0089'.
    """
    if matlab:  # Begin the numeration in the filenames with 1.
        num += 1
    len_num = len(str(num))
    str_num = ''
    for i in range(n_digs - len_num):
        str_num += '0'
    str_num += str(num)
    return str_num

In [ ]:
def find_out_dir(**kwargs):
    """Return a string that contains a route of a directory.
    
    With replace ('bool') equal to False, this will create a new
    directory if dirname ('string') exists in <s3out>/<parentdir>/
    
    From blendjupyter.ipynb (Blender kernel)."""
    s3out = kwargs.pop('s3out',
                       os.path.join(os.path.expanduser('~'),
                                    's3-out'))
    dirname = kwargs.pop('dirname', 'blend-phg-set-0001')
    parentdir = kwargs.pop('parentdir', 'scanner')
    path = kwargs.pop('path', os.path.join(s3out, parentdir, dirname))
    replace = kwargs.pop('replace', False)
    if not replace:  # Then, create a new direcctory with a different
                     # name.
        while True:
            if not os.path.exists(path):
                print('Make directory:', path)
                os.makedirs(path)
                break
            else:
                str_num = path.split('-')[-1]
                num_dir = int(str_num)
                num_dir += 1
                str_num_dir = num_str_zeros(num_dir, len(str_num))
                path = '-'.join(path.split('-')[:-1]) + '-' + str_num_dir
    return path

In [ ]:
def generate_lores_set(obj, cft, leds_prc, kx, ky, w, h,
                       dkx, dky, p, q, **kwargs):
    """Return an array ('numpy.ndarray') with the sequence of Lo-Res
    images which are also stored.
    
    Each image only contains the amplitude information."""
    prefix = kwargs.pop('prefix', 'lores-img_')
    extension = kwargs.pop('extension', '.tif')
    outpath = kwargs.pop('outpath',
                         find_out_dir(dirname='lores-set-0001',
                                      parentdir='microscope'))
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(generate_lores_set.__name__, list(kwargs.keys())[-1]))
    obj_ft = np.fft.fftshift(np.fft.fft2(obj))
    img_seq_lores = list()
    for i in range(leds_prc ** 2):
        kxc = round_half_up((w + 1) / 2.0 + kx[i] / dkx)
        kyc = round_half_up((h + 1) / 2.0 + ky[i] / dky)
        kyl = round_half_up(kyc - (p - 1) / 2.0);
        kyh = round_half_up(kyc + (p - 1) / 2.0);
        kxl = round_half_up(kxc - (q - 1) / 2.0);
        kxh = round_half_up(kxc + (q - 1) / 2.0);
        img_seq_lores_ft = (p / h) ** 2
        img_seq_lores_ft *= obj_ft[kyl - 1:kyh, kxl - 1:kxh]
        img_seq_lores_ft *= cft
        img_lores = np.absolute(np.fft.ifft2(np.fft.ifftshift(img_seq_lores_ft)))
        img_seq_lores.append(img_lores)
        str_i = num_str_zeros(i, len(str(leds_prc ** 2)), True)
        filename = prefix + str_i + extension
        img_path = os.path.join(outpath, filename)
        # scipy.misc.imsave() rescales the dynamic range of the
        # pixel values [3]. Add cmin and cmax as parameters to
        # scipy.misc.toimage() as follow to prevent the rescaling.
        # Before, giving the np.uint8 to dtype('type') to img_lores
        # (numpy.ndarray) there was no effect by the rescaling.
        # 
        # [3] Lippens, S. (2016). Saving of images in scipy and
        # preventing dynamic range rescaling.
        scipy.misc.toimage(img_lores, cmin=0, cmax=255).save(img_path)
        print('Saved Lo-Res image:', img_path)
    return np.array(img_seq_lores)

In [ ]:
def get_centered_img_subset(leds_prc, leds2use, **kwargs):
    """Return a personalized set with the images' indexes to use.
    
    In the lighting process with the LED grid, the lower incidence
    angle belongs to the central lamp. This function allow to select a
    square subset of images with a shifted center.
    
    ONLY USE IT with an external Lo-Res image set. For odd or even
    numbers of rows, the result is the same.
    """
    matlab = kwargs.pop('matlab', False)
    # To approximate even numbers.
    lower = kwargs.pop('lower', False)
    # Begin the selection down ('int') rows down.
    down = kwargs.pop('down', 0)
    # Begin the selection right ('int') columns right.
    right = kwargs.pop('right', 0)
    shift = kwargs.pop('shift', 0)
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(get_idx_img2use.__name__, list(kwargs.keys())[-1]))
    # The even numbers are approximated to the nearest lower integer.
    if lower:
        if leds_prc % 2 == 0:
            leds_prc -= 1
        if leds2use % 2 == 0:
            leds2use -= 1
    else:
        if leds_prc % 2 == 0:
            leds_prc += 1
        if leds2use % 2 == 0:
            leds2use += 1
    arr = np.arange(leds_prc ** 2)
    if matlab:  # In Matlab, the indexes begin in 1.
        arr = np.arange(1, leds_prc ** 2 + 1)
    arr = np.split(arr, leds_prc)  # Here, arr is type 'list'.
    # Convert arr ('list') to 'numpy.ndarray' to use advanced
    # indexing, for example, arr[0:2, 9:89].
    arr = np.array(arr)
    limit_l_up = int((leds_prc - leds2use) / 2)
    limit_r_down = leds_prc - limit_l_up
    if shift != 0:  # Roll arr ('numpy.ndarray'). A negative number
                    # rotate it to left and positive number, to
                    # right.
        arr = np.roll(arr, shift)
    arr = arr.flat
    arr[:shift] = -1  # Replace with 1's the shifted positions.
    arr = np.reshape(arr, (leds_prc, leds_prc))
    arr = arr[limit_l_up - down :limit_r_down - down, limit_l_up + right:limit_r_down + right]
    my_set = list(arr.flat)
    return my_set

In [ ]:
def get_set_from_folder(subsel, **kwargs):
    """Return an array with the Lo-Res images as arrays and a list
    with the indexes of np.zeros() in the l_imgs ('list')."""
    s3out = kwargs.pop('s3out',
                       os.path.join(os.path.expanduser('~'),
                                    's3-out'))
    dirname = kwargs.pop('dirname', 'lores-set-0001')
    # Path of the directory that contains the low-resolution image
    # set.
    path = kwargs.pop('path', os.path.join(s3out,
                                           'microscope',
                                           dirname))
    prefix = kwargs.pop('prefix', 'lores-img_')
    extension = kwargs.pop('extension', '.tiff')
    lensuffix = kwargs.pop('lensuffix', len(str(max(subsel))))
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(get_set_from_folder.__name__, list(kwargs.keys())[-1]))
    l_imgs = list()
    l_idx_zeros = list()
    for i in range(len(subsel)):
        if subsel[i] != -1:
            filepath = os.path.join(path, prefix + num_str_zeros(subsel[i], lensuffix) + extension)
            try:
                img = Image.open(filepath)
                arr_img = np.array(img)
                print('Opened Lo-Res image:', filepath)
                img.close()
                l_imgs.append(arr_img)
            except IOError:
                print('No such file:', filepath)
                l_imgs.append(np.zeros((0, 0)))
                l_idx_zeros.append(i)
        else:
            l_imgs.append(np.zeros((0, 0)))
            l_idx_zeros.append(i)
    return np.array(l_imgs), l_idx_zeros